# Install and Import Dependencies

In [34]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encode and Calculate Sentiment

In [39]:
tokens = tokenizer.encode('The food was good', return_tensors='pt')
result = model(tokens)
result.logits
#int(torch.argmax(result.logits))+1

tensor([[-1.9499, -1.0195,  0.8797,  1.4231,  0.5120]],
       grad_fn=<AddmmBackward>)

# Collect Reviews

In [17]:
#r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
r = requests.get('https://www.imdb.com/title/tt9813792/reviews')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*text.*')
results = soup.find_all('div', {'class':regex})
reviews = [result.text for result in results]

In [18]:
reviews


["Very intriguing series. Good storytelling. Good characters as in most of them have common sense. Good horror tv. Gory as heck in parts. Lots of mysterious things to ponder. I very much enjoyed season 1 and hope they make another. There's not many good horror tv shows out there anymore. This is a good one imo.",
 '\n                    219 out of 244 found this helpful.\n                        \n                            Was this review helpful?  Sign in to vote.\n                        \n\nPermalink\n',
 "I wasn't sure what to expect from From. I've read everything from it's like Lost to it's a generic horror version of Lost. Lost is one of my favorite shows ever and the reviews for this have been mostly positive so I figured I'd give it a try and I'm glad I did. I was hooked from the first episode to the last. I'm so glad they already renewed this because there are so many unanswered questions still. That's probably the only negative thing I have say about his show, that it didn

# Load Reviews into DataFrame and Score

In [19]:
import numpy as np
import pandas as pd

In [20]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [21]:
df['review'].iloc[0]

"Very intriguing series. Good storytelling. Good characters as in most of them have common sense. Good horror tv. Gory as heck in parts. Lots of mysterious things to ponder. I very much enjoyed season 1 and hope they make another. There's not many good horror tv shows out there anymore. This is a good one imo."

In [22]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [23]:
sentiment_score(df['review'].iloc[1])

1

In [24]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [25]:
df

,review,sentiment
0,Very intriguing series. Good storytelling. Goo...,4
1,\n 219 out of 244 found thi...,1
2,I wasn't sure what to expect from From. I've r...,4
3,\n 46 out of 53 found this ...,1
4,I never heard of this until a few weeks ago un...,4
5,\n 13 out of 13 found this ...,1
6,"It's a slow burner, but it has me hooked. I wa...",4
7,\n 220 out of 273 found thi...,1
8,"I am really enjoying this series, really hangi...",4
9,\n 130 out of 165 found thi...,1


In [26]:
df['review'].iloc[48]

"I noticed 1 rating comments getting downvoted into oblivion. Doesn't mean they're wrong, I too am trying to like this show because the overall concept is very interesting. But the execution feels way off. Just too many plot points defying logic and common sense.Given the consequences of opening a door or window at night, THINK would any of you who have children allow ANY of them to sleep alone in their own room? OR would any adult with a working brain choose to live in a big house with dozens of strangers, moody teenagers, and psychologically challenged individuals who do drugs, smoke weed, drink, and at the same time TRUST THAT NONE OF THEM will open a window or a door at night?There are ways to handle these type of plot points so that it makes sense but this is what happens with lazy writing. And I know, most characters in horrors make bad decisions (I've seen the Progressive commercial).. but still!?Aside from that the overall tone feels off. Wouldn't it be better to keep the tensi